<a href="https://colab.research.google.com/github/kocsis-david/Assigment7-RAG/blob/main/Copy_of_AIT_RAG_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG + LLM Assessment

Your task is to create a Retrieval-Augmented Generation (RAG) system using a Large Language Model (LLM). The RAG system should be able to retrieve relevant information from a knowledge base and generate coherent and informative responses to user queries.

Steps:

1. Choose a domain and collect a suitable dataset of documents (at least 5 documents - PDFs or HTML pages) to serve as the knowledge base for your RAG system. Select one of the following topics:
   * latest scientific papers from arxiv.org,
   * fiction books released,
   * legal documentsor,
   * social media posts.

   Make sure that the documents are newer then the training dataset of the applied LLM. (20 points)

2. Create three relevant prompts to the dataset, and one irrelevant prompt. (20 points)

3. Load an LLM with at least 5B parameters. (10 points)

4. Test the LLM with your prompts. The goal should be that without the collected dataset your model is unable to answer the question. If it gives you a good answer, select another question to answer and maybe a different dataset. (10 points)

5. Create a LangChain-based RAG system by setting up a vector database from the documents. (20 points)

6. Provide your three relevant and one irrelevant prompts to your RAG system. For the relevant prompts, your RAG system should return relevant answers, and for the irrelevant prompt, an empty answer. (20 points)


In [1]:
!pip install transformers>=4.32.0 optimum>=1.12.0 > null
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ > null
!pip install -i https://pypi.org/simple/ bitsandbytes > null
!pip install spacy==3.7.4 typer==0.10.0 imageio==2.31.6 pillow==10.1.0
!pip install langchain > null
!pip install chromadb > null
!pip install sentence_transformers > null # ==2.2.2
!pip install unstructured > null
!pip install pdf2image > null
!pip install pdfminer.six > null
!pip install unstructured-pytesseract > null
!pip install unstructured-inference > null
!pip install faiss-gpu > null
!pip install pikepdf > null
!pip install pypdf > null
!pip install accelerate
!pip install pillow_heif > null
!pip install -i https://pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of spacy to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install spacy==3.7.4 and typer==0.10.0 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested typer==0.10.0
    spacy 3.7.4 depends on typer<0.10.0 and >=0.3.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depend

In [2]:
from huggingface_hub import login
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, BitsAndBytesConfig
from textwrap import fill
from langchain.prompts import PromptTemplate
import locale
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores.utils import filter_complex_metadata # 'filter_complex_metadata' removes complex metadata that are not in str, int, float or bool format
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import bitsandbytes
import accelerate

In [3]:
login("")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
bb = BitsAndBytesConfig(load_in_8bit=True)
model_name="meta-llama/Meta-Llama-3-8B-Instruct"
#model_name="google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=bb,
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

gen_cfg = GenerationConfig.from_pretrained(model_name)
gen_cfg.max_new_tokens=512
gen_cfg.temperature=0.0000001 # 0.0 # For RAG we would like to have determenistic answers
gen_cfg.return_full_text=True
gen_cfg.do_sample=True
gen_cfg.repetition_penalty=1.11

pipe=pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=gen_cfg
)

llm = HuggingFacePipeline(pipeline=pipe)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Example of question without related context

In [33]:
template_gemma = """
<bos><start_of_turn>user
{text}<end_of_turn>
<start_of_turn>model
"""

template_llama3 = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

if "gemma" in model_name:
  template=template_gemma
else:
  template=template_llama3

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [34]:
text = "What is EUAT in training?"
result = llm(prompt.format(text=text))
print(fill(result.strip(), width=100))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>  What is EUAT in
training?<|eot_id|><|start_header_id|>assistant<|end_header_id|> EUAT stands for Emergency
Unannounced Training. It's a type of training exercise that is designed to test an individual's or
team's ability to respond to an emergency situation without prior notice.  In the context of
military, law enforcement, and other high-stress professions, EUATs are often used to evaluate an
individual's or unit's readiness to respond to unexpected scenarios, such as a sudden attack,
natural disaster, or crisis. The goal of an EUAT is to assess how well individuals or teams can
think on their feet, make quick decisions, and execute tasks under pressure.  During an EUAT,
participants may be presented with a simulated scenario, such as a mock terrorist attack, hostage
situation, or search and rescue operation. They must then respond accordingly, using their training
and skills to overcome challenges and achieve objectives.  EUA

With context

In [35]:
prompt_template_gemma = """
<bos><start_of_turn>user
Use the following context to Answer the question at the end. Do not use any other information. If you can't find the relevant information in the context, just say you don't have enough information to answer the question. Don't try to make up an answer.

{context}

Question: {question}<end_of_turn>

<start_of_turn>model
"""

prompt_template_llama3 = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Use the following context to Answer the question at the end. Do not use any other information. If you can't find the relevant information in the context, just say you don't have enough information to answer the question. Don't try to make up an answer.

{context}<|eot_id|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

In [36]:
promt_template=prompt_template_llama3
prompt = PromptTemplate(template=promt_template, input_variables=["context", "question"])

In [24]:
!wget -O document1.pdf --no-check-certificate "https://arxiv.org/pdf/2405.01269.pdf"
!wget -O document2.pdf --no-check-certificate "https://arxiv.org/pdf/2405.00726.pdf"
!wget -O document3.pdf --no-check-certificate "https://arxiv.org/pdf/2405.01205.pdf"
!wget -O document4.pdf --no-check-certificate "https://arxiv.org/pdf/2405.01260.pdf"
!wget -O document5.pdf --no-check-certificate "https://arxiv.org/pdf/2405.01266.pdf"
from langchain.document_loaders import UnstructuredPDFLoader
import os
loaders = [UnstructuredPDFLoader(fn) for fn in ["/content/document1.pdf","/content/document2.pdf","/content/document3.pdf", "/content/document4.pdf","/content/document5.pdf"]]

from langchain.text_splitter import RecursiveCharacterTextSplitter

chunked_pdf_doc = []

for loader in loaders:
    print("Loading raw document..." + loader.file_path)
    pdf_doc = loader.load()
    updated_pdf_doc = filter_complex_metadata(pdf_doc)
    print("Splitting text...")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
    documents = text_splitter.split_documents(updated_pdf_doc)
    chunked_pdf_doc.extend(documents)

len(chunked_pdf_doc)
embeddings = HuggingFaceEmbeddings()
db_pdf = FAISS.from_documents(chunked_pdf_doc, embeddings)

--2024-05-05 17:30:44--  https://arxiv.org/pdf/2405.01269.pdf
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.3.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2405.01269 [following]
--2024-05-05 17:30:44--  http://arxiv.org/pdf/2405.01269
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1359920 (1.3M) [application/pdf]
Saving to: ‘document1.pdf’

document1.pdf       100%[===================>]   1.30M  --.-KB/s    in 0.01s   

2024-05-05 17:30:44 (102 MB/s) - ‘document1.pdf’ saved [1359920/1359920]

--2024-05-05 17:30:44--  https://arxiv.org/pdf/2405.00726.pdf
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.67.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Perm

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [37]:
%%time
Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db_pdf.as_retriever(search_type="similarity_score_threshold", search_kwargs={'k': 4, 'score_threshold': 0.2}),
    chain_type_kwargs={
        "prompt": prompt,
    },
)



CPU times: user 2.39 ms, sys: 0 ns, total: 2.39 ms
Wall time: 2.26 ms


One question without context:

In [26]:
query = "What are the risk factors for NVIDIA?"
result = Chain_pdf({"query": query})
print(fill(result["result"].strip(), width=100))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='2) Devices : Various companies offer a variety of devices for monitoring human brain signals, utilizing both invasive and non-invasive methods to collect data. In non-invasive techniques, electrodes are positioned on the scalp without surgical intervention. This approach is commonly used due to its safety and ease of application. In contrast, invasive tech- niques involve a surgical process to place electrodes directly on or within the brain, providing more direct and often more detailed brain signal readings. Table III provides a compre- hensive overview of the most popular devices currently 

<|begin_of_text|><|start_header_id|>system<|end_header_id|>  Use the following context to Answer the
question at the end. Do not use any other information. If you can't find the relevant information in
the context, just say you don't have enough information to answer the question. Don't try to make up
an answer.  <|eot_id|><|start_header_id|>user<|end_header_id|>  What are the risk factors for
NVIDIA?<|eot_id|><|start_header_id|>assistant<|end_header_id|> Based on the provided context, I do
not have enough information to identify specific risk factors for NVIDIA. The context does not
provide any details about potential risks or threats that may affect NVIDIA. Therefore, I cannot
provide a list of risk factors for NVIDIA.


Questions with contest

In [27]:
query = "What is EEG-based brain-to-text communication?"
result = Chain_pdf.invoke(query)
print(fill(result['result'].strip(), width=100))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>  Use the following context to Answer the
question at the end. Do not use any other information. If you can't find the relevant information in
the context, just say you don't have enough information to answer the question. Don't try to make up
an answer.  EEG-based brain-to-text communication is a revolutionary field that explores using EEG
to decode brain signals into actual text. EEG measures electrical activity on the scalp, and
researchers are currently devising techniques to convert particular patterns into letters, words,
and even sentences. This technological advancement exhibits significant promise for persons who
experience speech or motor disabilities, offer- ing a direct communication channel that bypasses
traditional methods. BCIs play a pivotal role in EEG-based brain-to-text communication. BCIs bridge
the gap between the brain and external devices by interpreting brain signals to facilitate con- trol
or communicat

In [38]:
query = "What are the Privacy Concerns in the realm of EEG-to-text technology"
result = Chain_pdf.invoke(query)
print(fill(result['result'].strip(), width=100))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>  Use the following context to Answer the
question at the end. Do not use any other information. If you can't find the relevant information in
the context, just say you don't have enough information to answer the question. Don't try to make up
an answer.  F. Ethical Considerations  Privacy Concerns: Privacy concerns constitute a critical
ethical challenge in the realm of EEG-to-text technology. EEG data, which might disclose private and
confidential information  about a person’s mental condition, thoughts, or intentions, presents
substantial concerns regarding privacy [33]. Preserv- ing the confidentiality and security of this
data is of utmost importance, as breaches could result in unauthorized access and exploitation of
personal information. This is particularly relevant in the context of BCIs, where EEG data is used
for direct communication or control. The risk of eavesdropping or hacking into these systems poses a
serious 

In [32]:
query="What is EUAT in training?"
result = Chain_pdf.invoke(query)
print(fill(result['result'].strip(), width=100))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>  Use the following context to Answer the
question at the end. Do not use any other information. If you can't find the relevant information in
the context, just say you don't have enough information to answer the question. Don't try to make up
an answer.  binary classification problem) before starting the second phase of training where we ap-
plied the EUAT. The implementation of the training pipeline and ad- ditional information to ensure
the reproducibility of results are pro- vided in the supplemental material.  (5)  Table 2:
Comparison of EUAT against the baselines using different evaluation metrics and considering four
benchmarks.  Benchmark  Baseline  uA  uAUC  Corr. w/ res. Wasser. dist.  ECE  Error
ResNet50/ImageNet  EUAT CE Calibration DEUP Ensemble CALS CE+PE  0.804 0.749 0.743 0.580 0.745 0.746
0.755  0.878 0.812 0.807 0.591 0.809 0.809 0.786  0.655 0.546 0.538 0.291 0.536 0.542 0.530  0.301
0.220 0.193 0.030 0.213 0

In [ ]:
#Older homework for a previous course
from dotenv import load_dotenv
import streamlit as st
from PyPDF2 import PdfReader
from langchain import HuggingFaceHub
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import  HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback


def main():
    load_dotenv()
    st.set_page_config(page_title="Ask your PDF")
    st.header("Ask your PDF")

    # upload file
    pdf = st.file_uploader("Upload your PDF", type="pdf")

    # extract the text
    if pdf is not None:
        pdf_reader = PdfReader(pdf)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()

        # split into chunks
        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=400,
            chunk_overlap=10,
            length_function=len
        )
        chunks = text_splitter.split_text(text)

        # create embeddings
        embeddings =HuggingFaceInstructEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
        knowledge_base = FAISS.from_texts(chunks, embeddings)

        # show user input
        user_question = st.text_input("Ask a question about your PDF:")
        if user_question:
            docs = knowledge_base.similarity_search(user_question)

            llm = HuggingFaceHub(repo_id="google/flan-t5-xxl")
            chain = load_qa_chain(llm, chain_type="stuff")
            with get_openai_callback() as cb:
                response = chain.run(input_documents=docs, question=user_question)
                print(cb)

            st.write(response)


if __name__ == '__main__':
    main()